![Banner logo](https://raw.githubusercontent.com/CitrineInformatics/community-tools/master/templates/fig/citrine_banner_2.png "Banner logo")

## Programmatic Data Operations

*Authors: Zach del Rosario (zdelrosario@citrine.io)*

The purpose of this exercise is to give you some tools to work with data *programmatically*; that is, using a programming language. While you can carry out many data operations by hand or with spreadsheet programs, you will see that doing things programmatically is extremely powerful. 

### Learning Outcomes
By working through this notebook, you will be able to:

- Build self-sufficiency by consulting documentation to learn new programming concepts
- Initialize the Citrination application programming interface (API) and obtain data
- Inspect Python objects with `dir()`
- Learn some basics of *data wrangling*
- Use DataFrame operations in the Python package `pandas`
<!-- solution-begin -->
- Learn the basics of *featurization* to support training machine learning models (Not sure this one will make the cut -- ZDR)
<!-- solution-end -->

(Note: This is a *scavenger hunt*! You will have to follow the links below to finish these examples.)

### Q1: Setting up the Citrination client
Using the [learn-citrination](https://github.com/CitrineInformatics/learn-citrination/blob/master/citrination_api_examples/clients_sequence/1_data_client_api_tutorial.ipynb) workbook as an example, set up the citrination client below.

In [1]:
## Import some relevant packages
import os
# Scientific computation
import numpy as np
import pandas as pd

# Workshop-specific tools
from workshop_utils import pifs2df, ddir

# Third-party packages
from citrination_client import CitrinationClient
from citrination_client import PifSystemReturningQuery, PifSystemQuery
from citrination_client import DataQuery, DatasetQuery, DatasetReturningQuery, ChemicalFieldQuery
from citrination_client import PropertyQuery, FieldQuery
from citrination_client import ChemicalFilter, Filter

## TASK: Initialize the client below...
## You will need to provide `client` as a python object
# solution-begin
site = "https://citrination.com" # site you want to access; we'll use the public site
client = CitrinationClient(
    api_key = os.environ.get('CITRINATION_API_KEY'), 
    site = site
)
# solution-end

### Q2: Obtaining a known dataset 
Search [citrination datasets](https://citrination.com/datasets) for the "Agrawal IMMI" dataset, find its `ID`, and load the data into memory. 

In [33]:
# task-begin
dataset_id = 1      # TASK: Identify the proper dataset id, use this below
# task-end
# solution-begin
dataset_id = 150670 
# solution-end
search_client = client.search
query_agrawal = \
    PifSystemReturningQuery(
        size=500, 
        query=DataQuery(
            dataset=DatasetQuery(
                id=Filter(equal=str(dataset_id))
            )
        )
    )

## Perform checks
query_result = search_client.pif_search(query_agrawal)
print("Found {} PIFs in dataset.".format(query_result.total_num_hits, dataset_id))
print("(Should be 437 PIFs)")

Found 437 PIFs in dataset.
(Should be 437 PIFs)


Citrination stores data in [physical information files](http://citrineinformatics.github.io/pif-documentation/) (PIFs). 
 
### Reading a query result 
(Turn the PIFs above into rectangular data)

In [3]:
# query_result has a few useful attributes
dir(query_result)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_convert_to_dictionary',
 '_get_object',
 '_hits',
 '_max_score',
 '_took',
 '_total_num_hits',
 'as_dictionary',
 'hits',
 'max_score',
 'took',
 'total_num_hits']

In [4]:
# The __stuff__ attributes are python built-ins; the other
# other attributes are features provided by the object.
# total_num_hits was used above to count the number of search hits
# hits gives the content of the query hits
query_result.hits[:5]

In [5]:
# The query hits are themselves objects; we'll need to access *their* attributes as well
ddir(query_result.hits[0]) # Helper function filters names with "_" prefix

['as_dictionary',
 'dataset',
 'dataset_version',
 'extracted',
 'extracted_path',
 'id',
 'score',
 'system',
 'updated_at']

### Q3: Extract the PIFs
Complete the following code by *extracting* the PIFs from the `query_result`. You will need to use a loop or list comprehension.

In [6]:
# It's not at all obvious from the name, but the `system` attribute returns the actual PIF
query_result.hits[0].system

## TASK: Build a list of list of all the PIF's in query_result, and store it in `pifs`
# task-begin
pifs = []
# task-end
# solution-begin
pifs = [x.system for x in query_result.hits]
# solution-end
# Utility function will tabularize PIFs into a plot-able form
df_data = pifs2df(pifs)
df_data.head(5)

,Area Proportion of Isolated Inclusions,Normalizing Temperature,Tempering Temperature,Cooling Rate for Through Hardening,Area Proportion of Inclusions Deformed by Plastic Work,Fatigue Strength,Through Hardening Temperature,Cooling Rate for Tempering,Area Proportion of Inclusions Occurring in Discontinuous Array,Diffusion time,Diffusion Temperature,Sample Number,Carburization Temperature,Carburization Time,Reduction Ratio (Ingot to Bar),Through Hardening Time,Tempering Time,Quenching Media Temperature (for Carburization)
0,0.01,870.0,550.0,8.0,0.02,451.0,845.0,24.0,0.0,0.0,30.0,228.0,30.0,0.0,530.0,30.0,60.0,30.0
1,0.03,870.0,550.0,8.0,0.04,631.0,855.0,24.0,0.0,0.0,30.0,193.0,30.0,0.0,510.0,30.0,60.0,30.0
2,0.01,870.0,600.0,8.0,0.03,406.0,845.0,24.0,0.0,0.0,30.0,233.0,30.0,0.0,610.0,30.0,60.0,30.0
3,0.0,865.0,550.0,24.0,0.1,433.0,865.0,24.0,0.0,0.0,30.0,22.0,30.0,0.0,1740.0,30.0,60.0,30.0
4,0.01,870.0,650.0,8.0,0.03,385.0,845.0,24.0,0.0,0.0,30.0,240.0,30.0,0.0,610.0,30.0,60.0,30.0


## DataFrames

A `DataFrame` is a data structure provided by Pandas. In contrast with `lists` (which we saw in the previous exercise), DataFrames are explicitly designed to facilitate data analysis. Accordingly, they provide a number of helpful features that aid in data analysis and operations.

A `DataFrame` is a *rectangular* representation of data -- it consists of rows and columns. Each *row* represents an *observation* -- a single instance of data. Each *column* represents a *variable* -- a particular attribute of the observation. For instance, we have loaded some alloy data into the DataFrame `df_data` -- here each row is an alloy, and each column is some physical property of that alloy.

Below, we will use pandas functions to study the alloy data using DataFrame operations.

### Q4: Inspecting a DataFrame
Consult the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html) (it might be useful to use a page search) and use some basic calls on `df_data` to answer the following questions:

- What are the *last* five observations in the DataFrame?
- How many rows are in `df_data`? How many columns?
- How can you select the column "Normalizing Temperature"?
- How can you select the columns "Normalizing Temperature" and "Fatigue Strength"?

In [13]:
## Task: Show last five observations of df_data
# solution-begin
df_data.tail(5)
# solution-end

,Area Proportion of Isolated Inclusions,Normalizing Temperature,Tempering Temperature,Cooling Rate for Through Hardening,Area Proportion of Inclusions Deformed by Plastic Work,Fatigue Strength,Through Hardening Temperature,Cooling Rate for Tempering,Area Proportion of Inclusions Occurring in Discontinuous Array,Diffusion time,Diffusion Temperature,Sample Number,Carburization Temperature,Carburization Time,Reduction Ratio (Ingot to Bar),Through Hardening Time,Tempering Time,Quenching Media Temperature (for Carburization)
432,0.00,870.0,650.0,8.0,0.03,490.0,855.0,24.0,0.00,0.0,30.0,210.0,30.0,0.0,530.0,30.0,60.0,30.0
433,0.00,845.0,600.0,24.0,0.08,463.0,845.0,24.0,0.00,0.0,30.0,64.0,30.0,0.0,1740.0,30.0,60.0,30.0
434,0.00,870.0,550.0,8.0,0.10,592.0,855.0,24.0,0.00,0.0,30.0,148.0,30.0,0.0,820.0,30.0,60.0,30.0
435,0.02,885.0,30.0,0.0,0.06,245.0,30.0,0.0,0.02,0.0,30.0,6.0,30.0,0.0,825.0,0.0,0.0,30.0
436,0.00,870.0,550.0,8.0,0.02,526.0,855.0,24.0,0.00,0.0,30.0,141.0,30.0,0.0,530.0,30.0,60.0,30.0


In [14]:
## Task: Determine the number of rows and columns in df_data
# solution-begin
df_data.shape # rows, columns
# solution-end

(437, 18)

In [15]:
## Task: Select the column "Normalizing Temperature"
# solution-begin
df_data[["Normalizing Temperature"]].head()
# solution-end

,Normalizing Temperature
0,870.0
1,870.0
2,870.0
3,865.0
4,870.0


In [16]:
## Task: Select the columns "Normalizing Temperature" and "Fatigue Strength"
# solution-begin
df_data[["Normalizing Temperature", "Fatigue Strength"]].head()
# solution-end

,Normalizing Temperature,Fatigue Strength
0,870.0,451.0
1,870.0,631.0
2,870.0,406.0
3,865.0,433.0
4,870.0,385.0


These manipulations are simple, but they are bread-and-butter for studying new datasets.

## Wrangling Data
[Hadley Wickham](http://hadley.nz/) -- author of the `tidyverse` and data science superstar -- notes that "wrangling data is 80% boredom and 20% screaming". To give you a sense of why this stuff is hard (but hopefully avoid the screaming), I'm leaving one of the wrangling steps in the workflow here:

It's not obvious from the exercises above, but *there's an issue with these data*.

In [7]:
df_data.dtypes

Area Proportion of Isolated Inclusions                            object
Normalizing Temperature                                           object
Tempering Temperature                                             object
Cooling Rate for Through Hardening                                object
Area Proportion of Inclusions Deformed by Plastic Work            object
Fatigue Strength                                                  object
Through Hardening Temperature                                     object
Cooling Rate for Tempering                                        object
Area Proportion of Inclusions Occurring in Discontinuous Array    object
Diffusion time                                                    object
Diffusion Temperature                                             object
Sample Number                                                     object
Carburization Temperature                                         object
Carburization Time                                 

All of the entries are objects, not numbers! We'll need to convert these to numeric values. The following slightly-mysterious call will cast every column of `df_data` to a numeric type and modify the DataFrame.

In [18]:
df_data = df_data.apply(pd.to_numeric)

Let's check the data types again:

In [19]:
df_data.dtypes

Area Proportion of Isolated Inclusions                            float64
Normalizing Temperature                                           float64
Tempering Temperature                                             float64
Cooling Rate for Through Hardening                                float64
Area Proportion of Inclusions Deformed by Plastic Work            float64
Fatigue Strength                                                  float64
Through Hardening Temperature                                     float64
Cooling Rate for Tempering                                        float64
Area Proportion of Inclusions Occurring in Discontinuous Array    float64
Diffusion time                                                    float64
Diffusion Temperature                                             float64
Sample Number                                                     float64
Carburization Temperature                                         float64
Carburization Time                    

These are numbers we can work with!

## Basic DataFrame Operations

With the numerical issues above sorted out, we can carry out *quantitative* operations on the dataframe. One useful thing we can do is compute a set of *summaries* on the data using `describe()`.

In [21]:
df_data.describe()

,Area Proportion of Isolated Inclusions,Normalizing Temperature,Tempering Temperature,Cooling Rate for Through Hardening,Area Proportion of Inclusions Deformed by Plastic Work,Fatigue Strength,Through Hardening Temperature,Cooling Rate for Tempering,Area Proportion of Inclusions Occurring in Discontinuous Array,Diffusion time,Diffusion Temperature,Sample Number,Carburization Temperature,Carburization Time,Reduction Ratio (Ingot to Bar),Through Hardening Time,Tempering Time,Quenching Media Temperature (for Carburization)
count,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000
mean,0.007712,872.299771,536.842105,10.654462,0.047181,552.903890,737.643021,20.814645,0.003391,4.843936,123.699844,219.000000,128.855835,40.502059,923.629291,25.949657,65.080092,35.491991
std,0.010418,26.212073,164.101963,7.841437,0.031093,186.630528,280.036541,8.072207,0.008240,15.700076,267.128933,126.295289,281.743539,126.924697,576.617020,10.263824,21.477650,19.419277
min,0.000000,825.000000,30.000000,0.000000,0.000000,225.000000,30.000000,0.000000,0.000000,0.000000,30.000000,1.000000,30.000000,0.000000,240.000000,0.000000,0.000000,30.000000
25%,0.000000,865.000000,550.000000,8.000000,0.020000,448.000000,845.000000,24.000000,0.000000,0.000000,30.000000,110.000000,30.000000,0.000000,590.000000,30.000000,60.000000,30.000000
50%,0.000000,870.000000,600.000000,8.000000,0.040000,505.000000,845.000000,24.000000,0.000000,0.000000,30.000000,219.000000,30.000000,0.000000,740.000000,30.000000,60.000000,30.000000
75%,0.010000,870.000000,650.000000,8.000000,0.070000,578.000000,855.000000,24.000000,0.000000,0.000000,30.000000,328.000000,30.000000,0.000000,1228.000000,30.000000,60.000000,30.000000
max,0.058000,930.000000,680.000000,24.000000,0.130000,1190.000000,865.000000,24.000000,0.050000,70.200000,903.333000,437.000000,930.000000,540.000000,5530.000000,30.000000,120.000000,140.000000


These summaries include things like the `mean` and standard deviation (`std`), as well as quartiles of the data. These give us a sense of *typical* values; for instance, we can see that a large fraction of observations have a zero-"Diffusion time", but at least one observation has a value `> 70`.

### Q5: Basic data operations
Once more, use the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html) to learn how to do the following tasks:

- Select only those rows for which "Diffusion time" is greater than 70
- Sort df_data in descending order by "Fatigue Strength" and return the top 10
- Take the average of "Normalizing Temperature" and "Tempering Temperature" and add the column "avg_temp" (You may need to Google how to do this one!)

In [29]:
## TASK: Select rows for which "Diffusion time" > 70
# solution-begin
df_data[df_data["Diffusion time"] > 70]
# solution-end

,Area Proportion of Isolated Inclusions,Normalizing Temperature,Tempering Temperature,Cooling Rate for Through Hardening,Area Proportion of Inclusions Deformed by Plastic Work,Fatigue Strength,Through Hardening Temperature,Cooling Rate for Tempering,Area Proportion of Inclusions Occurring in Discontinuous Array,Diffusion time,Diffusion Temperature,Sample Number,Carburization Temperature,Carburization Time,Reduction Ratio (Ingot to Bar),Through Hardening Time,Tempering Time,Quenching Media Temperature (for Carburization)
27,0.000,930.0,160.0,0.0,0.080,1144.0,30.0,0.5,0.000,70.2,895.812,429.0,930.0,340.2,690.0,0.0,120.0,60.0
40,0.000,930.0,160.0,0.0,0.050,1030.0,30.0,0.5,0.010,70.2,895.812,433.0,930.0,340.2,240.0,0.0,120.0,140.0
53,0.004,930.0,160.0,0.0,0.025,876.0,30.0,0.5,0.000,70.2,895.812,420.0,930.0,529.8,460.0,0.0,120.0,140.0
75,0.010,930.0,200.0,0.0,0.070,1120.0,30.0,0.5,0.000,70.2,895.812,424.0,930.0,499.8,690.0,0.0,120.0,60.0
97,0.000,930.0,160.0,0.0,0.025,900.0,30.0,0.5,0.004,70.2,895.812,419.0,930.0,499.8,390.0,0.0,120.0,60.0
146,0.004,930.0,200.0,0.0,0.025,873.0,30.0,0.5,0.000,70.2,895.812,423.0,930.0,499.8,460.0,0.0,120.0,60.0
160,0.000,930.0,200.0,0.0,0.025,898.0,30.0,0.5,0.004,70.2,895.812,425.0,930.0,499.8,390.0,0.0,120.0,60.0
178,0.010,930.0,160.0,0.0,0.070,1068.0,30.0,0.5,0.000,70.2,895.812,421.0,930.0,529.8,690.0,0.0,120.0,140.0
214,0.000,930.0,200.0,0.0,0.080,1104.0,30.0,0.5,0.000,70.2,895.812,435.0,930.0,340.2,690.0,0.0,120.0,60.0
260,0.008,930.0,200.0,0.0,0.017,882.0,30.0,0.5,0.004,70.2,895.812,437.0,930.0,340.2,530.0,0.0,120.0,60.0


In [30]:
## TASK: Sort by "Fatigue Strength" in descending order, take the top-10
# solution-begin
df_data.sort_values(by = "Fatigue Strength", ascending = False).head(10)
# solution-end

,Area Proportion of Isolated Inclusions,Normalizing Temperature,Tempering Temperature,Cooling Rate for Through Hardening,Area Proportion of Inclusions Deformed by Plastic Work,Fatigue Strength,Through Hardening Temperature,Cooling Rate for Tempering,Area Proportion of Inclusions Occurring in Discontinuous Array,Diffusion time,Diffusion Temperature,Sample Number,Carburization Temperature,Carburization Time,Reduction Ratio (Ingot to Bar),Through Hardening Time,Tempering Time,Quenching Media Temperature (for Carburization)
141,0.000,930.0,160.0,0.0,0.070,1190.0,30.0,0.5,0.01,15.0,850.000,397.0,930.0,540.0,860.0,0.0,120.0,140.0
27,0.000,930.0,160.0,0.0,0.080,1144.0,30.0,0.5,0.00,70.2,895.812,429.0,930.0,340.2,690.0,0.0,120.0,60.0
291,0.000,930.0,160.0,0.0,0.070,1139.0,30.0,0.5,0.01,15.0,850.000,391.0,930.0,139.8,860.0,0.0,120.0,60.0
185,0.000,930.0,160.0,0.0,0.070,1124.0,30.0,0.5,0.01,15.0,850.000,394.0,930.0,469.8,860.0,0.0,120.0,60.0
75,0.010,930.0,200.0,0.0,0.070,1120.0,30.0,0.5,0.00,70.2,895.812,424.0,930.0,499.8,690.0,0.0,120.0,60.0
177,0.000,930.0,200.0,0.0,0.070,1110.0,30.0,0.5,0.01,15.0,850.000,400.0,930.0,469.8,860.0,0.0,120.0,60.0
214,0.000,930.0,200.0,0.0,0.080,1104.0,30.0,0.5,0.00,70.2,895.812,435.0,930.0,340.2,690.0,0.0,120.0,60.0
231,0.000,930.0,160.0,0.0,0.060,1089.0,30.0,0.5,0.00,34.8,895.517,407.0,930.0,450.0,690.0,0.0,120.0,60.0
294,0.010,930.0,160.0,0.0,0.070,1086.0,30.0,0.5,0.00,34.8,895.517,415.0,930.0,100.2,690.0,0.0,120.0,60.0
263,0.008,930.0,160.0,0.0,0.017,1082.0,30.0,0.5,0.00,15.0,850.000,395.0,930.0,469.8,440.0,0.0,120.0,60.0


In [32]:
## TASK: Average "Normalizing Temperature" and "Tempering Temperature" into the column "avg_tmp", return the head
# solution-begin
df_data.assign(avg_tmp = 0.5 * (df_data["Normalizing Temperature"] + df_data["Tempering Temperature"])).head()
# solution-end

,Area Proportion of Isolated Inclusions,Normalizing Temperature,Tempering Temperature,Cooling Rate for Through Hardening,Area Proportion of Inclusions Deformed by Plastic Work,Fatigue Strength,Through Hardening Temperature,Cooling Rate for Tempering,Area Proportion of Inclusions Occurring in Discontinuous Array,Diffusion time,Diffusion Temperature,Sample Number,Carburization Temperature,Carburization Time,Reduction Ratio (Ingot to Bar),Through Hardening Time,Tempering Time,Quenching Media Temperature (for Carburization),avg_tmp
0,0.01,870.0,550.0,8.0,0.02,451.0,845.0,24.0,0.0,0.0,30.0,228.0,30.0,0.0,530.0,30.0,60.0,30.0,710.0
1,0.03,870.0,550.0,8.0,0.04,631.0,855.0,24.0,0.0,0.0,30.0,193.0,30.0,0.0,510.0,30.0,60.0,30.0,710.0
2,0.01,870.0,600.0,8.0,0.03,406.0,845.0,24.0,0.0,0.0,30.0,233.0,30.0,0.0,610.0,30.0,60.0,30.0,735.0
3,0.00,865.0,550.0,24.0,0.10,433.0,865.0,24.0,0.0,0.0,30.0,22.0,30.0,0.0,1740.0,30.0,60.0,30.0,707.5
4,0.01,870.0,650.0,8.0,0.03,385.0,845.0,24.0,0.0,0.0,30.0,240.0,30.0,0.0,610.0,30.0,60.0,30.0,760.0
